In [ ]:
CREATE OR REPLACE TEMPORARY VIEW dns
USING org.apache.spark.sql.parquet
OPTIONS (
  path "s3a://data/dns/"
)

In [ ]:
CREATE OR REPLACE TEMPORARY VIEW dns_history
USING org.apache.spark.sql.parquet
OPTIONS (
  path "s3a://test/temp/dns_history/"
)

In [ ]:
CREATE OR REPLACE TEMPORARY VIEW dns_report
AS
SELECT count(DISTINCT user) users, count(*) resolutions, date
FROM dns
WHERE date >= date_add(current_date(), -3)
GROUP BY date

In [ ]:
INSERT OVERWRITE DIRECTORY "s3a://test/temp/dir1"
USING parquet
SELECT * FROM dns_history
WHERE date < date_add(current_date(), -3)

In [ ]:
CREATE OR REPLACE TEMPORARY VIEW dir1
USING org.apache.spark.sql.parquet
OPTIONS (
  path "s3a://test/temp/dir1"
)

In [ ]:
INSERT OVERWRITE DIRECTORY "s3a://test/temp/dns_history/"
USING parquet
SELECT * FROM dns_report
UNION ALL
SELECT * FROM dir1

In [ ]:
SELECT "${env:S3_ACCESS_KEY}" AS ENV, 1